# Assignment 4
## Implementation: Hidden Markov Model

In [2]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mv_normal
import matplotlib.mlab as mlab
from scipy.stats import chi2
from matplotlib.patches import Ellipse

We start off by loading the training data:

In [4]:
train_data = np.loadtxt('data/EMGaussian.train')
test_data = np.loadtxt('data/EMGaussian.test')

We start by implementing the alpha-beta recursion

In [ ]:
def alpha(A, pi):
    pass

def beta(A, pi):
    pass

We will define a helper function that will help us compute the Gaussian pdf. This method will be used to plot the contours as well.

In [5]:
def mv_gauss(X, Y, mu, cov):
    sigma_x = np.sqrt(cov[0,0])
    sigma_y = np.sqrt(cov[1,1])
    sigma_xy = np.sqrt(cov[0,1])
    
    mu_x = mu[0]
    mu_y = mu[1]
    
    return mlab.bivariate_normal(X, Y, sigma_x, sigma_y, mu_x, mu_y, sigma_xy)

In [6]:
# Credit to: 
# http://www.nhsilbert.net/source/2014/06/bivariate-normal-ellipse-plotting-in-python/
def plot_cov_ellipse(cov, pos, volume=.5, ax=None, fc='none', ec=[0,0,0], a=1, lw=1):
    """
    Plots an ellipse enclosing *volume* based on the specified covariance
    matrix (*cov*) and location (*pos*). Additional keyword arguments are passed on to the 
    ellipse patch artist.

    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        volume : The volume inside the ellipse; defaults to 0.5
        ax : The axis that the ellipse will be plotted on. Defaults to the 
            current axis.
    """

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    kwrg = {'facecolor':fc, 'edgecolor':ec, 'alpha':a, 'linewidth':lw}

    # Width and height are "full" widths, not radius
    width, height = 2 * np.sqrt(chi2.ppf(volume,2)) * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, **kwrg)

    ax.add_artist(ellip)

### Implementation of the K-means algorithm

In [13]:
class K_means:
    
    def __init__(self, k=4, n_dims=2):
        self.k = k
        self.n_dims = n_dims
        
    def train(self, train_data):
        # Initialize the cluster means
        self.means = np.random.rand(self.k, self.n_dims) * np.max(train_data, axis=0)
        n_iter = 0
        
        # Matrix where each row is a z_n assignment vector associated with a data point
        old_Z = np.zeros(shape=(train_data.shape[0], self.k))
        self.Z = np.zeros(shape=(train_data.shape[0], self.k))
        
        while(not self._converged(old_Z, n_iter)):
            old_Z = np.array(self.Z)
            self.Z = np.zeros(shape=(train_data.shape[0], self.k))
            
            # First phase, we evaluate the value of the latent cluster assignment variables
            for i, train_point in enumerate(train_data):                
                distances = np.linalg.norm(self.means - train_point, axis=1)**2
                self.Z[i][np.argmin(distances)] = 1 
            
            # Second phase, the values of the cluster means are computed
            self.means = self.Z.T.dot(train_data) / np.sum(self.Z.T, axis=1).reshape(self.k, 1)
            
            n_iter += 1
        
    def assign_cluster(self, data):
        
        # Will hold the cluster that each data point belongs to
        clusters = np.zeros(data.shape[0], dtype=int)
        
        for i, x in enumerate(data):            
            distances = np.linalg.norm(self.means - x, axis=1)**2
            clusters[i] = np.argmin(distances)
        
        return clusters
            
    
    # Helper function that checks the convergence of the K-means algorithm
    def _converged(self, old_Z, n_iter):
        if n_iter == 0:
            return False
        elif np.array_equal(old_Z, self.Z):
            return True
        else:
            return False

### EM algorithm for a Gaussian mixture with general covariance matrix

In [23]:
class EM_GMM:
    
    def __init__(self, k=4, n_dims=2):
        self.k = k
        self.n_dims = n_dims
        
    def train(self, train_data, means, clusters, MAX_ITER = 100):
        # We start off by initializing our gaussian mixture parameters with the parameters given to us
        self.means = means
        self.covs = [np.eye(self.n_dims)] * self.k
        
        # compute the sample covariance of each cluster
        for i in xrange(self.k):
            self.covs[i] = np.cov(train_data[np.where(clusters==i)[0],:], rowvar=False) 

        # posterior probabilities or the weights N x K matrix
        self.taus = np.zeros(shape=(train_data.shape[0], self.k))
        self.pi = np.bincount(clusters) / clusters.shape[0]        
        n_iter = 0
        
        while(n_iter < MAX_ITER):            
            # E step
            for i in xrange(self.k):
                self.taus[:, i] = self.pi[i] * mv_normal.pdf(train_data, self.means[i], self.covs[i], True)
            
            # normalize the taus to get posterior probabilities
            self.taus = (self.taus.T / np.sum(self.taus, axis=1)).T
            
            # M step            
            # Compute the new means and covariance matrices
            for i in xrange(self.k):
                tau_sum = np.sum(self.taus[:, i])
                
                # First the mean for cluster i
                self.means[i] = (np.sum(self.taus[:, i].reshape(self.taus.shape[0], 1) * train_data, axis=0) / tau_sum)
                
                distance = train_data - self.means[i]
                self.covs[i] = (distance.T.dot(self.taus[:, i].reshape(self.taus.shape[0], 1) * distance) / tau_sum)
                self.pi[i] = tau_sum / train_data.shape[0]

                n_iter += 1
    
    def assign_cluster(self, data):
        taus = np.zeros(shape=(data.shape[0], self.k))
        for i in xrange(self.k):
                taus[:, i] = self.pi[i] * mv_normal.pdf(data, self.means[i], self.covs[i], True)
                
        clusters = np.zeros(data.shape[0], dtype=int)
        for i, x in enumerate(data):
            clusters[i] = np.argmax(taus[i, :])
        return clusters
    
    def normalized_log_likelihood(self, data):
        like = np.zeros(shape=(data.shape[0], self.k))
        for i in xrange(self.k):
                like[:, i] = self.pi[i] * mv_normal.pdf(data, self.means[i], self.covs[i], True)
        
        loglike = np.log(np.sum(like, axis=1))
        loglike = np.sum(loglike) / data.shape[0]
        
        return loglike
                